In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import spotipy 
import csv
import json

df= pd.read_csv('top_21000_in_year.csv',nrows=10001) 
df

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
cid ="" #username for the spotify developer interface 
secret = "" # 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False

features = []
moods=[]
for i in range(0,len(df['URL']),50):    
    audio_features = sp.audio_features(df['URL'][i:i+50])
    for track in audio_features:
        features.append(track)

from pandas import DataFrame
df_features=DataFrame.from_dict(features, orient='columns', dtype= None)
df_features.drop(['analysis_url','track_href', 'type', 'uri','time_signature','id'], axis=1, inplace=True)
df= pd.concat([df, df_features], axis=1)
#df.to_csv('top_10000_in_year_with_features.csv', encoding='utf-8')

In [ ]:
features =[]
moods=[]
for i in range(0,len(df['URL']),50):    
    audio_features = sp.audio_features(df['URL'][i:i+50])
    for track in audio_features:
        energy= track['energy']
        valence= track['valence']
        moods.append(set_mood(float(energy),float(valence)) )
        features.append(track)

df_moods=DataFrame(moods)

In [ ]:
df['key'].describe()

In [ ]:
def population_means():    
    int_columns= ['key','Streams','acousticness','danceability','duration_ms','energy','loudness','liveness','valence','tempo']
    for column in int_columns :
        mean = df[column].mean()
        print("Mean of " + column + " in the total list:" + str(mean) )

population_means()

In [ ]:
def population_histograms():
    int_columns= ['key','acousticness','danceability','duration_ms','energy','loudness','liveness','valence','tempo']
    for column in int_columns :
        plt.hist(df[column], bins=100)
        #plt.axvline(df[column].mean(), color='r', linestyle='dashed', linewidth=1)
        plt.xlabel(column)
        plt.ylabel('Count')
        plt.show()
population_histograms()

In [ ]:
import scipy.stats as stats
import statsmodels.api as sm

sm.qqplot(df['energy'])
plt.show()

In [ ]:
cols = list(df.columns.values)
print(cols)
df= df[['Position', 'Track Name', 'Artist', 'URL','Streams', 'duration_ms','mode', 'acousticness', 'danceability', 'energy',  'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']]
df.head()

In [ ]:
audio_features=df.loc[:,"Streams":"valence"]
corr_features=audio_features.corr()
corr_features

In [ ]:
audio_features=df.loc[:,"Streams":"valence"]
corr_features=audio_features.corr()
corr_features
import seaborn as sns
fig = plt.figure(figsize=(8,8))
sns.heatmap(corr_features); 

In [ ]:
df_sample= df.head(500)
df_sample

In [ ]:
def sample_histograms():
    int_columns= ['acousticness','danceability','duration_ms','energy','loudness','liveness','valence','tempo']
    for column in int_columns :
        plt.hist(df_sample[column], bins=30)
        plt.axvline(df_sample[column].mean(), color='r', linestyle='dashed', linewidth=1)
        plt.xlabel(column)
        plt.ylabel('Count')
        plt.show()

sample_histograms()


In [ ]:
df.describe()

In [ ]:
df_sample.describe()

In [ ]:
from scipy import stats
#since acousticness doesn't have a normal distribution, this feature can't be tested by T-test
int_columns= ['danceability','duration_ms','energy','loudness','valence','tempo']
for column in int_columns :
    pop_mean= np.mean(df[column])
    print ('Population mean of ', column ,' :            ',pop_mean)
    pop_std=np.std(df[column])
    print ('Population standard deviation of ', column ,' : ', pop_std)
    sample_mean=np.mean(df_sample[column])
    print ('Top 500- Sample mean of ', column ,' : ', sample_mean)
    sample_std=np.std(df_sample[column])
    print ('Top 500- Sample standard deviation of ', column ,' : ', sample_std)
    print('T-test results for ',column,' :',stats.ttest_1samp(df_sample[column],pop_mean))
    print('--------------------------------------------------------------------------------')